## 머신러닝의 유형 
- 지도학습 : 정답을 포함하고 있는 데이터를 이용해 분류와 수치 예측
    - 종류 : 선형회귀, 결정트리, 랜덤 포레스트 등등
- 비지도학습 : 정답을 미포함한 데이터를 이용해 문제의 설명 특징도출, 정보나 패턴을 탐색하고자 할 때
    - 종류 : 군집화, 차원 축소 기법, 연관 관계 분석 등
- 강화학습 : 행동 중 보상을 최대화 하는 행동 혹은 행동순서를 선택 , 보상을 통하여 학습을 진행

## 머신러닝의 절차
1. 데이터 수집
2. 데이터 전처리
    - 결측값 처리, 중복데이터 삭제, 분석변수처리 등
3. EDA(탐색적 데이터 분석)
    - 데이터 시각화, 기초 통계량 이해 등
4. 모델선택
    - 회기모델, 분류모델 구분 /  성능을 최적화 하는 모델 
5. 평가

# 훈련데이터와 평가데이터 나누기 
- 일반적으로 훈련데이터가 80% , 평가데이터가 20% 정도의 비율을 차지한다
- 교차검증이 아닌 검증데이터를 따로 사용하기 위해 훈련데이터를 한번 더 훈련과 검증으로 나누기도 한다
- 훈련데이터를 전처리한 과정을 똑같이 평가데이터도 전처리 해줘야함

### train_test_split 사용하기
- from sklearn.model_select import train_test_split
- x_train,y_train,x_test,y_test = train_test_split(data, test_size = )

* 완전 무작위로 train과 test를 나누지 않고 어떠한 특성에 의존해 균등한 분포를 가지게 하기위해선 
- stratify= data['특정컬럼'] 의 옵션을 추가 (특정 컬럼의 값이 적절한 비율로 나눠지도록 test와 train이 나눠진다)

# 데이터 전처리 

### 결손값 처리하기 - pandas 
- isnull() : 결손값 확인
- dropna() : 결손값 삭제하기 
- fillna(특정값) : 결손값 특정값으로 대체하기 

### 결손값 처리하기 - sklearn
- from sklearn.impute import SimpleImputer
- imputer = SimpleImputer(strategy='median') : 변환기 객체 생성
- imputer.fit_transform(data) : 변환

### 인코딩 하기 - pandas
- 레이블 인코딩 : pd.factorize()
- 원핫 인코딩 : pd.get_dummies()

### 인코딩 하기 - sklearn
- 레이블 인코딩
    - from sklearn.preprocessing import OrdinalEncoder
- 원핫 인코딩
    - from sklearn.preprocessing import OneHotEncoder

### 스케일링 하기 - sklearn
- MinMax : 최대값을 1, 최소값을 0으로 정규화(이상치에 영향을 받는다)
    - from sklearn.preprocessing import MinMaxScaler
- Standard : 평균 0, 표준편차 1을 만족하는 정규분포
    - from sklearn.preprocessing import StandardScaler

    * 참고사항
```
- 변환기의 학습(fit)은 train데이터에서만 사용해야한다
- test데이터에는 train데이터로 학습한 변환기를 이용해 transform만 실시해주면 된다
     - test데이터도 학습(fit)하면 기존 train 데이터만 가지고 학습했을때와 조건이 달라진다. 
     - test에도 train에 적용되었던 동일한 조건을 적용하기 위해서 train에만 fit
```


     * 나만의 변환기
```
# 1. 아래 형식을 그대로 가져가기
class 나만의변환기(BaseEstimator, TransformerMixin):
    def __init__(self):
        # 초기값 만들어주기
    
    def fit(self, X, y=None):
        # 변환단계에서 준비되어야 할 내용 작성
        
    def transform(self, X):
        # 변환될 내용 작성


```

### 파이프라인 사용하기 
- 변환을 한번에
- from sklearn.pipeline import Pipeline

    * pipeline ex)
 
    num_pipeline =  Pipeline([
                   ("변환기이름1", SimpleImputer(strategy='median')), 
                   ("변환기이름2", CombinedAttributesAdder()),
                   ("변환기이름3", StandardScaler()), 
               ])
               
    housing_num_tr = num_pipeline.fit_transform(data)

### ColumnTransformer 사용하기
- 컬럼에 특성에 따라 변환을 해주는 형식이 다를때 여러개의 파이프라인을 사용한다
- 이 파이프라인을 거친 데이터를 다시 하나의 데이터로 합쳐줘야한다
- ColumnTransformer : 위의 두 과정을 한번에 실시
- from sklearn.compose import ColumnTransformer

     * ColumnTransformer ex)
     수치형 데이터 = [수치형데이터의 컬럼들]
     범주형 데이터 = [범주형데이터의 컬럼들]
     
    full_pipeline = ColumnTransformer([
                    ("num", num_pipeline, 수치형 데이터), 
                    ("cat", cat_pipeline(), 범주형 데이터)  
                ])
    full_data_prepared = full_pipeline.fit_transform(전체데이터)

# 모델 선정하기 

    * 모델 훈련 및 예측 시키기 ex)
    모델 선정 : model = LinearRegression()
    모델 훈련 : model.fit(x_train,y_train)
    예측 하기 : model.predict(x_test)

## 모델 검증하기
- 모델 학습중에는 test데이터를 이용해 모델을 평가할 수 없다
1. 검증데이터를 사용해 평가한다
2. 교차검증을 실시해 평가한다

1. 검증데이터 사용 
    - 학습 전 train데이터 일부를 검증데이터로 분류
    - 학습 된 모델을 이용해 검증데이터를 예측한 후 y값과 비교
    - 데이터가 적을 때 사용하기 힘들다는 단점이 있음1

2. 교차검증 사용 
    - 홀드아웃, k-fold, LOOCV등의 검증방법이 있음
    - sklearn에서 제공하는 cross_val_score를 사용하면 k-fold형식의 교차검증을 실시
        - from sklearn.model_selection import cross_val_score

```
* cross_val_score()

    from sklearn.model_selection import cross_val_score
    
    score = cross_val_score(model, x_train,y_train, scoring="평가지표",cv=나눌그룹의수)
    
    -> score는 cv의 개수만큼 list의 요소로 나타남, 평균을 내야 최종 score로 생각할 수 있다
```

    * cross_validate를 사용하면 한번에 여러개의 평가지표를 사용할 수 있다
      또 결과값으로 fit_time,score_time,test_score이 딕셔너리의 형태로 출력된다
      return_train_score 파라미터를 True 로 설정하면 train_score도 출력된다

## 하이퍼파라미터 튜닝하기
- 하이퍼파라미터 : 사용자가 지정해줘야하는 파라미터
- 파라미터를 바꿔줄 때 마다 검증을 하지 않고 Grid Search를 이용해 최적의 모델을 찾음
    - from sklearn.model_selection import GridSearchCV
- 파라미터와 경우의 수를 딕셔너리 형태로 정의해 줘 경우의수 중 최적의 조합을 찾음

```
    * GridSearch()
from sklearn.model_selection import GridSearch

parms = {'파라미터' ; [경우1, 경우2, 경우3]}

gridsearch_model = GridSearchCV(model, parms, scoring=평가지표, cv=교차검증개수, n_jobs=-1)
```

- GridSearch를 사용하면
- best_estimator_ 에 훈련된 모델이 저장되어있음
- best_params_ 를 이용해 가장좋은 파라미터를 확인
- cv_results_ 를 사용하면 각 파라미터일때의 점수도 확인가능
- best_estimator_에 feature_importances_를 사용하면 각 컬럼의 가중치를 확인가능

- RandomizedSearch를 사용하기 
    - gridsearch와 비슷하지만, 파라미터의 목록을 전달하는것이 아니고 랜덤하거나 균등하게 나눠져있는 범위를 전달
    - n_iter만큼의 랜덤뽑기를 한 후 최적의 파라미터를 조합을 출력함
    - 그 외의것은 gridsearch와 동일

```
    *RandomizedSearch()
from sklenarn.model_selection import RandomizedSearch

prams = {'파라미터1' : umiform(0.0001,0.001),
         '파라미터2' : randint(20,50)}
         
randomsearch_model = RandomizedSearch(model, parms, n_iter=횟수, scoring=평가지표)
```

# 평가

## 과대적합 
- train데이터에 대한 점수는 높지만 test데이터에 대한 평가는 저조한 경우

#### 과대적합 해결방법 
1. train 데이터의 수를 더 늘린다
2. train 데이터의 이상치를 제거한다
3. 모델의 복잡도를 낮춘다
4. train에 불필요한 특성은 제거한다

## 과소적합
- train,test의 평가가 모두 저조하거나, test의 점수가 더 높은 경우

#### 과소적합 해결방법
1. 파라미터가 더 많은 복잡한 모델을 선택한다
2. 더 좋은 특성을 추가한다
3. 모델의 제약을 줄여준다